## Q1: Implement a trie tree

In this question, you will write Python code that can take a set/list/tuple of strings and insert them into a trie tree and lookup whether a specific word/string is present in the trie tree.

### Theoretical pondering

Two main approaches to building trees, you might recall from class, are making separate Tree and Node classes, or only making a Node class. Which method do you think is a better fit for trie trees, and why? **Justify your reasoning in around 100 words.** You will use your chosen approach throughout the assignment, so don't rush this question.

I chose to implement separate Node and Tree classes for several reasons.  

1. Following the abstraction principle of object-oriented programming, it's better to isolate entities/objects whenever possible. This concept is known as modularity, and it's a best practice to follow it closely whenever possible—doing so, allowing for editing a block/object/entity once instead of going around the code looking for all the instances, the same functionality was implemented. For example, my node class in the first question has attributes of character, children, terminal, and words only. Later, in question 3, I realized that I needed a count attribute that signifies the number of times the node is crossed. Given my Node+Tree implementation, I simply added that attribute to the Node class and then used it whenever I wanted within the Tree class through inheritance. Meanwhile, if I had implemented only a Tree class, I would have needed to edit functionalities within the insert method to adapt to that now the children of self.root have a new attribute (count). This would have gone against the abstraction and modularity principles and sacrificed readability as a result.

2. Additionally, having each node as a separate object/entity means that the program doesn't have to load and cache the WHOLE Tree class with all of its methods EVERY time a node instance is created. While python loads the class, and then only passes the attributes called/needed and discards the rest. That process still takes time, even if tiny.

In [38]:
# VERSION 0 - QUESTION 1
class Node:
    """
    This class represents one node of a trie tree. I only store a character at each node and NOT a word.

    Parameters
    ----------  
        character : char
        The inserted character to be housed at the current node
    """

    def __init__(self, character):
        self.character = character
        
        #I used a hash table here (a dict) to achieve a constant time lookup for nodes associated with any key
        #structure: {char0:char_node0, char1:char_node1, char2:char_node2, ...}

        #a dict of a all DIRECT child nodes of the current node
        self.children = {}

        #an indicator whether there's an inserted word that ends at the current node (housing a terminal character or not)
        #this attribute is used while looking up to inform the program whether we hit the end of a word or not
        #this helps diffrentiate between looking up a prefix (undesired lookup) vs a complete word (desired lookup)
        self.terminal = False
        
        #the number of words that we'd expect stemming from the current node
        #this attribute is incremented whenever insert passes through the current node while inserting a word
        #notice here the current node does NOT have to be the first char/node in the insertee word. being an ancestor/component is enough
        self.words = 0
        
class Trie:
    """
    This class represents the entirety of a trie tree.
    
    Parameters
    ----------  
        word_list : list
        List of strings to be inserted into the trie upon creation.
    Methods
    -------
    insert(self, word)
        Inserts a word into the trie, creating nodes as required.
    lookup(self, word)
        Determines whether a given word is present in the trie.
    """

    def __init__(self, word_list):
        """
        Creates the Trie instance, inserts initial words if provided.   

        Parameters
        ----------
        word_list : list
            List of strings to be inserted into the trie upon creation.
        """
        #inheriting all methods and attributes of the parent class: Node
        super().__init__()
        #creating the root node, initialized with Node, using the inhereted Node class
        self.root = Node(None)

        #inserting the words into the trie
        for word in word_list:
            self.insert(word)
    
    def insert(self, word):
        """Inserts a word into the trie, creating missing nodes on the go.
        
        Parameters
        ----------
        word : str
            The word to be inserted into the trie.
        """
        length = len(word)
        current_node = self.root
        current_node.words += 1
        #normalizing the input by converting it to lowercase
        word = word.lower()

        #going over each character in the input word and checking whether there exists a node with the same character attribute or not
        #if such node exists, then we just add 1 to its words attribute count and go to the next character. meanwhile, if such node doesn't
        #exist, we create it
        #AFTER inserting all the character in the input word, we update the LAST node's terminal attribute as True, marking that it represents
        #an end to a word
        for insertee_char in word:
            children_chars = list(current_node.children.keys())

            #print(current_node.character, current_node.words, children_chars)
            #print(insertee_char)

            #make the current node adopt a new node if the insertee_char isn't already a child
            if insertee_char not in children_chars:
                current_node.children[insertee_char] = Node(insertee_char)
            
            current_node = current_node.children[insertee_char]
            current_node.words += 1
            
        #update the terminal of the current word to True when we know that we just finished inserting a word into the tree
        current_node.terminal = True
    
    def lookup(self, word):
        """Determines whether a given word is present in the trie.
        
        Parameters
        ----------
        word : str
            The word to be looked-up in the trie.
            
        Returns
        -------
        bool
            True if the word is present in trie; False otherwise.
        """
        #normalizing the input words to ignore uppercase letter
        word = word.lower()
        current_node = self.root
        word_found = ""

        #going over the characters in the lookup word and finding whether they exist within the trie or not
        #if we find that they all exist IN ORDER and the last node's terminal attribute is True, then we return True
        #in all other cases (as when we find a prefix) not a word, we return False
        for char in word:
            children_chars =  list(current_node.children.keys())
            #print(current_node.character, current_node.words, children_chars)
            
            if char in children_chars:
                #print(char, "found")
                current_node = current_node.children[char]
                word_found += current_node.character
            
            elif char not in children_chars:
                #print(char, "one char not found")
                return False
        
        #word_found == word
        return current_node.terminal

# This is Namárië, JRRT's elvish poem written in Quenya
wordbank = "مُحَمَّد Ai! laurië lantar lassi súrinen, yéni unótimë ve rámar aldaron! Yéni ve lintë yuldar avánier mi oromardi lisse-miruvóreva Andúnë pella, Vardo tellumar nu luini yassen tintilar i eleni ómaryo airetári-lírinen. Sí man i yulma nin enquantuva? An sí Tintallë Varda Oiolossëo ve fanyar máryat Elentári ortanë, ar ilyë tier undulávë lumbulë; ar sindanóriello caita mornië i falmalinnar imbë met, ar hísië untúpa Calaciryo míri oialë. Sí vanwa ná, Rómello vanwa, Valimar! Namárië! Nai hiruvalyë Valimar. Nai elyë hiruva. Namárië!".replace("!", "").replace("?", "").replace(".", "").replace(",", "").replace(";", "").split()

trie = Trie(wordbank)
# be careful about capital letters!
assert trie.lookup('oiolossëo') == True
# this is a prefix, but also a word in itself
assert trie.lookup('an') == True
# this is a prefix, but NOT a word
assert trie.lookup('ele') == False
# not in the wordbank
assert trie.lookup('Mithrandir') == False

In [39]:
#existing
assert trie.lookup('lantar') == True
assert trie.lookup('ve') == True
#my name in arabic
assert trie.lookup('مُحَمَّد') == True

#partially existing
assert trie.lookup('sdasdasdasdasd') == False
assert trie.lookup('Mithrasssnssssdir') == False

## Q2: The computational complexity of tries

- `Trie.insert(self, word)`: The computational complexity for this method is proportional to the number of words inserted and the length of each word. Since we loop over each character (C) for each inserted word (W), then at the worst case, the computational complexity evaluates to O(C*W). And given that c is bounded by 26 then we can regard it as a constant. Then, there isn't significant advantage in implementing my `Node` class to house a word instead of a character. Thus, the worse case computational complexity for inserting evaluates to O(n), where represesnts (W) the number of words inserted into the trie. For a binary tree, inserting would have also taken O(n).
-  `Trie.lookup(self, word)`: The computational complexity for this method is proportional to the length of the word to be looked up. Here, we loop over each character within the inserted word to check whether it has a matching node. At each character, we compare the nodes to the children. Since I used a hash table here so we have retrieval time of O(1). Then, at the worst case, the computational complexity evaluates to O(N). Notice that a binary tree wouldn't have allowed hashing which would have brought the time complexity up to O(n) for each lookup andn since we do that for each character then this would have evaluated to O(N*n). But since, N is upper bounded by 26 then the overall complexity boils down to O(n).

Here,the advantages of tries is printing in alphabetic orders, getting prefixes and allowing hashing easily. The height here is dependent on the length of the longest word. The maximum height is length of the longest word and the maximum level length is 26. Then the maximum number of nodes is represented by 26*length_longest_word. The worst case here isn't really common and the advantages of trie trees outweigh the non exisiting cost as neither the time or space complexity here are worse than binary trees. On the contrary, trie trees will have an average runtime of O(k) and a best runtime of O(1)

## Q3: Print a dictionary in alphabetical order.


Notice that here, the time and space (proportional to the height) complexities for both approaches in this case (recursive vs iterative with stack) are the same: O(n) where n is the total number of nodes in the trie tree.- 

### Added attributes
- `Trie.found_words`: Stores all of the char values found while traversing. Initially, the travie(node, prefix) method stores chars, word fragmnents, prefixes, and valid words found while traversing. But then, alphabetical_list cleans the lists to only include valid words found (inserted) within the trie
### Added methods  
- `Trie.travie(self, node, prefix)`: Traverses the trie tree recursively preorder, appending ALL the nodes (character) values within the trees to self.found_words
- `Trie.alphabetical_list(self)`: Delivers the content of the trie in alphabetical order

In [40]:
# VERSION 1 - QUESTION 3

class Node:
    """
    This class represents one node of a trie tree.
    
    Parameters
    ----------  
    character : char
        The inserted character to be housed at the current node
    """
    def __init__(self, character):
        self.character = character
        
        #I used a hash table here (a dict) to achieve a constant time lookup for nodes associated with any key
        #structure: {char0:char_node0, char1:char_node1, char2:char_node2, ...}

        #a dict of a all DIRECT child nodes of the current node
        self.children = {}
        self.children_lst = []

        self.terminal = False
        
        #the number of words that we'd expect stemming from the current node
        #notice here the current node does NOT have to be the first char/node in the insertee word. being an ancestor/component is enough
        self.words = 0
        
class Trie:
    """
    This class represents the entirety of a trie tree.
    
    Parameters
    ----------  
    word_list : list
        List of strings to be inserted into the trie upon creation.  
    Methods
    -------
    insert(self, word)
        Inserts a word into the trie, creating nodes as required.
    lookup(self, word)
        Determines whether a given word is present in the trie.
    travie(self, node, prefix)
        Traverses the trie tree recursively preorder, appending ALL the nodes (character) values within the trees to self.found_words
    alphabetical_list(self)
        Delivers the content of the trie in alphabetical order
    """

    #add docstring and comment
    def __init__(self, word_list):
        """
        Creates the Trie instance, inserts initial words if provided.   
        Parameters
        ----------
        word_list : list
            List of strings to be inserted into the trie upon creation.
        """
        super().__init__()
        self.root = Node(None)

        for word in sorted(word_list):
            self.insert(word)
        
        #a list of all the found node character values while traversing using travie. later, alphabetical_list cleans this list to include 
        #valid words that exist withing the trie only, rather than all node values and word fragments
        self.found_words = []
    
    def insert(self, word):
        """Inserts a word into the trie, creating missing nodes on the go.
        
        Parameters
        ----------
        word : str
            The word to be inserted into the trie.
        """
        length = len(word)
        current_node = self.root
        current_node.words += 1
        word = word.lower()

        for insertee_char in word:
            children_chars = list(current_node.children.keys())

            #print(current_node.character, current_node.words, children_chars)
            #print(insertee_char)

            #make the current node adopt a new node if the insertee_char isn't already a child
            if insertee_char not in children_chars:
                current_node.children[insertee_char] = Node(insertee_char)
                current_node.children_lst.append(insertee_char)
            
            current_node = current_node.children[insertee_char]
            current_node.words += 1
            
        #update the terminal of the current word to True when we know that we just finished inserting a word into the tree
        current_node.terminal = True
    
    def lookup(self, word):
        """Determines whether a given word is present in the trie.
        
        Parameters
        ----------
        word : str
            The word to be looked-up in the trie.
            
        Returns
        -------
        bool
            True if the word is present in trie; False otherwise.
            
        Notes
        -----
        Your trie should ignore whether a word is capitalized.
        E.g. trie.insert('Prague') should lead to trie.lookup('prague') = True
        """
        word = word.lower()
        current_node = self.root
        word_found = ""

        for char in word:
            children_chars =  list(current_node.children.keys())
            #print(current_node.character, current_node.words, children_chars)
            
            if char in children_chars:
                #print(char, "found")
                current_node = current_node.children[char]
                word_found += current_node.character
            
            elif char not in children_chars:
                #print(char, "one char not found")
                return False
        
        #word_found == word
        return current_node.terminal

    def travie(self, node, prefix):
        """
        Traverses the trie tree recursively preorder, appending ALL the nodes (character) values within the trees to self.found_words
        Afterwards, these character values are cleaned to remove non-word values
        
        Parameters
        ----------
        node : Node
            The node at which to start traversing. Default: self.root
        prefix : str
            The char value of the node where we start traversing
        """
        
        #base case: the current node doesn't have any children (leaf/sterile node)
        #break the stack when the current node is sterile (doesn't have any more children)
        if len(node.children_lst) < 0:
            #in this case, we don't have to return anything, but we need to break the stack to visit the unvisited nodes so we return an
            #empty string just to break the recursive call of travie
            return ""

        #since we can't add (contacenate) a none type and a string, I had to check whether each current node's character value is None (root)
        #or not. If it is, then we append the prefix to the found words. Otherwise, we append the prefix + the current node's character value
        
        #IMPORTANT: Notice here that since I can't capture the node character value while traversing with return as this a recursive fucntion
        #that has restricted returns, I, instead, capture the traversing stream into the found_words attribute 
        else:
            if node.character == None: 
                self.found_words.append(prefix)
            else:
                #append the input prefix of the initial node and the character value of the current node to the found_words attribute
                self.found_words.append(prefix + node.character)
            
            #since this is not a binary tree, we have to traverse/loop over the ALL the children of each node at each level from LEFT to RIGHT
            #thus, this approach is a combination of recursion and iteration, but not by choice as indicated above. I simply couldn't think
            #of a way to make this purely recursive or purely iterartive
            for child_node in node.children.values():
                #if we're at the root node, then traverse until you reach the bottom leaf/sterile node
                if node.character == None:
                    self.travie(child_node, prefix)
                #otherwise, traverse the nodes on the other levels
                else:
                    self.travie(child_node, prefix + node.character)
        
    def alphabetical_list(self):
        """
        Delivers the content of the trie in alphabetical order.
        
        Returns
        ----------
        list
            returns all valied words found in the tree

            List of strings, all words from the trie in alphabetical order.
        """
        #call the preorder traversal method to update the self.found_words attribute with all the words found while traversing
        self.travie(self.root, "")
        #a new list for the verified words
        verified_words = []
        
        #looping over the strings found while traversing and adding only the words verified to be inserted/existing withing the trie
        #to the verified_words list
        for word in list(set(self.found_words)):
            if self.lookup(word):
                verified_words.append(word)

        #return the verified words found after traversing, sorted (python uses timsort which takes O(n*logn))    
        return sorted(verified_words)

In [41]:
wordbank = "Lorem ipsum dolor sit amet, consectetuer adipiscing elit. Duis pulvinar. Class aptent taciti sociosqu ad litora torquent per conubia nostra, per inceptos hymenaeos. Nunc dapibus tortor vel mi dapibus sollicitudin. Etiam quis quam. Curabitur ligula sapien, pulvinar a vestibulum quis, facilisis vel sapien.".replace(",", "").replace(".", "").split()

trie = Trie(wordbank)

assert trie.alphabetical_list() == ['a','ad','adipiscing','amet','aptent',
                                    'class','consectetuer','conubia',
                                    'curabitur','dapibus','dolor','duis',
                                    'elit','etiam','facilisis','hymenaeos',
                                    'inceptos','ipsum','ligula','litora',
                                    'lorem','mi','nostra','nunc','per',
                                    'pulvinar','quam','quis','sapien',
                                    'sit','sociosqu','sollicitudin','taciti',
                                    'torquent','tortor','vel','vestibulum']

## Q4: Find the k most common words in a speech.


### Added/edited attributes
- `Node.children_lst`: A list of all the children nodes for any given node. I created this node following the abstraction principles of OOP to avoid finding the character atrivute for each children of the current node whenever I am debugging or during a normal function call.
- `Node.count`: A count of the number of words that END at the current nodes. This extremely helpful for counting the inserted words into the trie
- `Trie.found_words`: Now this attribute is a dict (hash table) instead of a list to allow for counting the frequency of each word found while treaversing, with the helo of Node.count. The counting process happens within `Trie.insert(self, word)`
### Added methods  
- `Trie.k_most_common(self, k)`: Finds k words inserted into the trie most often

In [43]:
# VERSION 2 - QUESTION 4

class Node:
    """
    This class represents one node of a trie tree.
    
    Parameters
    ----------  
    character : char
        The inserted character to be housed at the current node
    """
    def __init__(self, character):
        self.character = character
        
        #I used a hash table here (a dict) to achieve a constant time lookup for nodes associated with any key
        #structure: {char0:char_node0, char1:char_node1, char2:char_node2, ...}

        #a dict of a all DIRECT child nodes of the current node
        self.children = {}
        self.children_lst = []

        self.terminal = False
        
        #the number of words that we'd expect stemming from the current node
        #notice here the current node does NOT have to be the first char/node in the insertee word. being an ancestor/component is enough
        self.words = 0

        #this count for the number of times this node would have been repeated if the properties of trie allowed that
        #aka: the count of the number of words ending at the current node
        self.count = 1
        
class Trie:
    """
    This class represents the entirety of a trie tree.
    
    Parameters
    ----------  
    word_list : list
        List of strings to be inserted into the trie upon creation.
    
    Methods
    -------
    insert(self, word)
        Inserts a word into the trie, creating nodes as required.
    lookup(self, word)
        Determines whether a given word is present in the trie.
    travie(self, node, prefix)
        Traverses the trie tree recursively preorder, appending ALL the nodes (character) values within the trees to self.found_words.
    k_most_common(self, k)
        Finds k words inserted into the trie most often.
    """

    def __init__(self, word_list):
        """
        Creates the Trie instance, inserts initial words if provided.
      
        Parameters
        ----------
        word_list : list
            List of strings to be inserted into the trie upon creation.
        """
        super().__init__()
        self.root = Node(None)

        for word in sorted(word_list):
            self.insert(word)
        
        #a hash table (dict) of all the strings found while traversing, along with their count
        #notice that this count depends on the new attribute, Node.count that signifies the number of time a words was repeated
        self.found_words = {}
    
    def insert(self, word):
        """Inserts a word into the trie, creating missing nodes on the go.
        
        Parameters
        ----------
        word : str
            The word to be inserted into the trie.


        NOTES:
        EDITED FOR QUESTION 4 to increment the counter attribute inside the node whenever a word is repeated
        """
        length = len(word)
        current_node = self.root
        current_node.words += 1
        word = word.lower()

        repeated = True
        for insertee_char in word:
            #print(current_node.character, current_node.words, children_chars)
            #print(insertee_char)

            #make the current node adopt a new node if the insertee_char isn't already a child
            if insertee_char not in current_node.children_lst:
                current_node.children[insertee_char] = Node(insertee_char)
                current_node.children_lst.append(insertee_char)
                repeated = False
            
            current_node = current_node.children[insertee_char]
            current_node.words += 1
        
        #VERY IMPORTANT:
        # if we didn't create any new node AND THE WORD, not just the terminal node, already exists then our inserted word is repeated!
        if repeated and self.lookup(word):
            #if current_node.character in ["to", "a", "be", "will", "on", "be", "will", "new", "in", "than", "a", "be"]:
                #print(word, current_node.character, "REPEATED!")
            current_node.count += 1
        #update the terminal of the current word to True when we know that we just finished inserting a word into the tree
        current_node.terminal = True
    
    def lookup(self, word):
        """Determines whether a given word is present in the trie.
        
        Parameters
        ----------
        word : str
            The word to be looked-up in the trie.
            
        Returns
        -------
        bool
            True if the word is present in trie; False otherwise.
            
        Notes
        -----
        Your trie should ignore whether a word is capitalized.
        E.g. trie.insert('Prague') should lead to trie.lookup('prague') = True
        """
        word = word.lower()
        current_node = self.root
        word_found = ""

        for char in word:
            children_chars =  list(current_node.children.keys())
            #print(current_node.character, current_node.words, children_chars)
            
            if char in children_chars:
                #print(char, "found")
                current_node = current_node.children[char]
                word_found += current_node.character
            
            elif char not in children_chars:
                #print(char, "one char not found")
                return False
        
        #word_found == word
        return current_node.terminal

    def travie(self, node, prefix):
        """
        EDIT: NOW ADDING THE WORD COUNT VALUE TO THE DICT WHILE TRAVERSING

        Traverses the trie tree recursively preorder, appending ALL the nodes (character) values within the trees to self.found_words
        Afterwards, these character values are cleaned to remove non-word values
        
        Parameters
        ----------
        node : Node
            The node at which to start traversing. Default: self.root
        prefix : str
            The char value of the node where we start traversing
        """
        #break the stack when the child is sterile
        #if node.terminal:
            #print(node.character, "TERMINAL!")

        if len(node.children_lst) < 0:
            return ""
        else:
            if node.character == None:
                self.found_words[prefix] = (node.count)
            else:
                self.found_words[prefix + node.character] = (node.count)
            
            for child_node in node.children.values():
                if node.character == None:
                    self.travie(child_node, prefix)
                else:
                    self.travie(child_node, prefix + node.character)
        
    def k_most_common(self, k):
        """Finds k words inserted into the trie most often.

        Parameters
        ----------
        k : int
            Number of most common words to be returned.

        Returns
        ----------
        list
            List of tuples.
            
            Each tuple entry consists of the word and its frequency.
            The entries are sorted by frequency.

        """
        #call the preorder traversal method to update the self.found_words attribute with all the words found while traversing
        self.travie(self.root, "")
        #a new dict for the verified words
        verified_words = {}
        
        #sorting the words ***alphabetically*** before parsing them and converting to tuples
        for word in sorted(list(set(self.found_words.keys()))):
            #checking if the string foudn while traversing is a verified/insered word that exists within the trie
            if self.lookup(word):
                verified_words[word] = self.found_words[word]
        
        #raising an error when k is bigger than the number of verified words available
        #if k > len(verified_words):
        #    return "k should be between 1 and len(input)"

        #sorting the verified words by the number of occureneces
        sorted_verified_words = {a: b for a, b in sorted(verified_words.items(), key=lambda item: item[1], reverse = True)}
        #returning the k most common words as a list of tuple in the format of [(word1_str, word1_count), (word2_str, word2_count)...]
        return list(sorted_verified_words.items())[:k]

In [44]:
from requests import get
speakers = ['Faruqi', 'Kennedy', 'King', 'Thunberg', 'Havel']
bad_chars = [';', ',', '.', '?', '!', '_', 
             '[', ']', ':', '“', '”', '"', '–', '-']

test_words = []

for speaker in speakers:
    
    # download and clean up the speech from extra characters
    speech_full = get(f'https://bit.ly/CS110-{speaker}').text
    just_text = ''.join(c for c in speech_full if c not in bad_chars)
    without_newlines = ''.join(c if (c not in ['\n', '\r', '\t']) else " " for c in just_text)
    just_words = [word for word in without_newlines.split(" ") if word != ""]

    #appending all the speeches to test_words for debugging
    test_words.append(just_words)

### A debugging adventure
Problem statement: My program's counter was off by exactly 1 for the following words ONLY:  `problematic_words = ["to", "a", "be", "will", "on", "be", "will", "new", "in", "than", "a", "be"]` and I spend a whole day debugging this so I thought it's valuable to include my thinking process here.

To find where the problem was, I took a systematic approach of investigating one problem at a time
1. First, I thought the problem was with the parsing from the speech but I checked that by parsing the speech myself and find that it's identical to the version parsed and given in the problem statement
2. Second, I realized the problem was with the counting the words' frequency and SPECIFICALLY with the words problematic_words then, I printed whenever a word of the problematic_words are incremented and I found that my program was nodes with the same terminal letter not repeating nodes in general
* for example, given "australia" and "a" these were counted as equivalent. to fix that, I used self.lookup as a condition within my counter
* then, my counter worked but it wasn't printing in alphabetic order so I fixed the input to k_most_common to be sorted alphabetically as Q4

The two functions below `normal_counter(word_lst, end)` and `show_difference(current_count, actual_count)` helped me compare my counter, using trie trees, to a simple counter with a hash table (dict)

In [45]:
def normal_counter(word_lst, end):
    """
    counts the words frequency using a simple hash table (dict) instead of a trie
    Notes:
        the goal of this fucntion is to provide a (comparison) reference to the frequency of the words I calculate within the trie
    """
    word_dict_counter = {}

    normalized_words = [word.lower() for word in word_lst]
    for word in sorted(normalized_words):
        if word not in word_dict_counter:
            word_dict_counter[word] = 1
        else:
            word_dict_counter[word] += 1

    sorted_dict_counter = {a: b for a, b in sorted(word_dict_counter.items(), key=lambda item: item[1], reverse = True)}
    return list(sorted_dict_counter.items())[:end]
    #return word_dict_counter

def show_difference(current_count, actual_count):
    """
    shows the output difference between my output and the desired output to help narrow down the problem to the problematic cases
    Notes:
        this function helped me find problematic_words = ["to", "a", "be", "will", "on", "be", "will", "new", "in", "than", "a", "be"]
        which was crucial in realizing the problem I had within my counter
    """
    diff = []
    for tuple in current_count:
        if tuple not in actual_count:
            print(tuple, "not found!")
            word = tuple[0]
            diff.append(tuple)
    if diff:
        return diff
    elif len(current_count) == len(actual_count):
        return "IDENTICAL"

In [46]:
"""
PROBLEMATIC_WORDS within each speech:

1st speech: to, a, be, will, new, on (+1 error)  
2nd speech: a, be, will, new, on (+1 error)   
3rd speech: a (+1 error)  
4th speech: be, in, a, than (+1 error)  
5th speech: a, be, (+1 error)  
"""

#other testcases with problematic words
words2 = "roll roll round round route toUr tour tO to to route route taw in an a in in".split()
words3 = "the and to in of is that this a people has are for we have racism black justice lives police".split() * 101

#comparing my output with the ideal output
for test in test_words:
    choice = test
    tree = Trie(choice)
    print(show_difference(tree.k_most_common(25), normal_counter(choice, 25)))

IDENTICAL
IDENTICAL
IDENTICAL
IDENTICAL
IDENTICAL


In [47]:
# Mehreen Faruqi - Black Lives Matter in Australia: https://bit.ly/CS110-Faruqi
# John F. Kennedy - The decision to go to the Moon: https://bit.ly/CS110-Kennedy
# Martin Luther King Jr. - I have a dream: https://bit.ly/CS110-King
# Greta Thunberg - UN Climate Summit message: https://bit.ly/CS110-Thunberg
# Vaclav Havel - Address to US Congress after the fall of Soviet Union: https://bit.ly/CS110-Havel

from requests import get
speakers = ['Faruqi', 'Kennedy', 'King', 'Thunberg', 'Havel']
bad_chars = [';', ',', '.', '?', '!', '_', 
             '[', ']', ':', '“', '”', '"', '–', '-']

for speaker in speakers:
    
    # download and clean up the speech from extra characters
    speech_full = get(f'https://bit.ly/CS110-{speaker}').text
    just_text = ''.join(c for c in speech_full if c not in bad_chars)
    without_newlines = ''.join(c if (c not in ['\n', '\r', '\t']) else " " for c in just_text)
    just_words = [word for word in without_newlines.split(" ") if word != ""]
    
    #print(len(just_words))
    #print()

    trie = Trie(just_words)
    if speaker == 'Faruqi':
        #print(trie.k_most_common(20), "\n")
        Faruqi = [('the', 60), ('and', 45), ('to', 39), ('in', 37), 
                  ('of', 34), ('is', 25), ('that', 22), ('this', 21), 
                  ('a', 20), ('people', 20), ('has', 14), ('are', 13), 
                  ('for', 13), ('we', 13), ('have', 12), ('racism', 12), 
                  ('black', 11), ('justice', 9), ('lives', 9), ('police', 9)]
        assert trie.k_most_common(20) == Faruqi
    
    elif speaker == 'Kennedy':
        #print(trie.k_most_common(21), "\n")
        Kennedy = [('the', 117), ('and', 109), ('of', 93), ('to', 63), 
                   ('this', 44), ('in', 43), ('we', 43), ('a', 39), 
                   ('be', 30), ('for', 27), ('that', 27), ('as', 26), 
                   ('it', 24), ('will', 24), ('new', 22), ('space', 22), 
                   ('is', 21), ('all', 15), ('are', 15), ('have', 15), ('our', 15)]
        assert trie.k_most_common(21) == Kennedy
    
    elif speaker == 'Havel':
        #print(trie.k_most_common(22), "\n")
        Havel = [('the', 34), ('of', 23), ('and', 20), ('to', 15), 
                 ('in', 13), ('a', 12), ('that', 12), ('are', 9), 
                 ('we', 9), ('have', 8), ('human', 8), ('is', 8), 
                 ('you', 8), ('as', 7), ('for', 7), ('has', 7), ('this', 7), 
                 ('be', 6), ('it', 6), ('my', 6), ('our', 6), ('world', 6)]
        assert trie.k_most_common(22) == Havel
    
    elif speaker == 'King':
        #print(trie.k_most_common(23), "\n")
        King = [('the', 103), ('of', 99), ('to', 59), ('and', 54), ('a', 37), 
                ('be', 33), ('we', 29), ('will', 27), ('that', 24), ('is', 23), 
                ('in', 22), ('as', 20), ('freedom', 20), ('this', 20), 
                ('from', 18), ('have', 17), ('our', 17), ('with', 16), 
                ('i', 15), ('let', 13), ('negro', 13), ('not', 13), ('one', 13)]
        assert trie.k_most_common(23) == King
    
    elif speaker == 'Thunberg':
        #print(trie.k_most_common(24), "\n")
        Thunberg = [('you', 22), ('the', 20), ('and', 16), ('of', 15), 
                    ('to', 14), ('are', 10), ('is', 9), ('that', 9), 
                    ('be', 8), ('not', 7), ('with', 7), ('i', 6), 
                    ('in', 6), ('us', 6), ('a', 5), ('how', 5), ('on', 5), 
                    ('we', 5), ('all', 4), ('dare', 4), ('here', 4), 
                    ('my', 4), ('people', 4), ('will', 4)]
        assert trie.k_most_common(24) == Thunberg
        
# Note: There are cleaner and more concise ways to write the code above, 
# but this way it should be easily understandable.

## Q5: Implement an autocomplete with a Shakespearean dictionary!

My autocomplete engine works as following:
1. First, traverses the tree recursively preorder to find all the words inserted that start with the given prefix. While traversing, it also retrieves the word count value (calculated while inserting) and saves it into the `words_cands` hash table (dict) that allows for O(1) retrieval time
2. Second, We simply return the the word with the maximum frequency with `words_cands`

Here, I could have implemented a heap datastructure (populated with O(n*logn)) that could have immediately given me the word with the maximum value within constant time. However, that just didn't come to my mind after I finished. Instead, I just used python's built in `max()` method by passing the dict values (count for each word) as a key. This function has a time complexity of O(n) as it has to check every element to decide upon the words with the max count. I acknowledge that heapq would have been better in this case as O(n*logn) is much better than O(n). However, I just didn't have time to rewrite my code to use heapq instead of a list.

### Added attributes
- NOTHING
### Added methods  
- `Trie.k_most_common(self, k)`: Finds k words inserted into the trie most often
- `lookup_prefix(self, prefix)`: Checks if a prefix exists in a trie
- `clean_found_words(self)`: Cleans the found words so that we end up with all the words inserted represented in self.found_words
- `autocomplete(self, prefix)`: Finds the most common word with the given prefix.

In [ ]:
# VERSION 3 - QUESTION 5

class Node:
    """
    This class represents one node of a trie tree.
    
    Parameters
    ----------  
        character : char
        The inserted character to be housed at the current node
    """
    def __init__(self, character):
        self.character = character
        
        #I used a hash table here (a dict) to achieve a constant time lookup for nodes associated with any key
        #structure: {char0:char_node0, char1:char_node1, char2:char_node2, ...}

        #a dict of a all DIRECT child nodes of the current node
        self.children = {}
        self.children_lst = []

        self.terminal = False
        
        #the number of words that we'd expect stemming from the current node
        #notice here the current node does NOT have to be the first char/node in the insertee word. being an ancestor/component is enough
        self.words = 0

        #this count for the number of times this node would have been repeated if the properties of trie allowed that
        #aka: the count of the number of words ending at the current node
        self.count = 1
        
class Trie:
    """
    This class represents the entirety of a trie tree.
    
    Parameters
    ----------  
    word_list : list
        List of strings to be inserted into the trie upon creation.   
    
    Methods
    -------
    insert(self, word)
        Inserts a word into the trie, creating nodes as required.
    lookup(self, word)
        Determines whether a given word is present in the trie.
    lookup_prefix(self, prefix)
    travie(self, node, prefix)
        Traverse the trie recursively preorder
    clean_found_words(self)
        Cleans the found words so that we end up with all the words inserted represented in self.found_words
    k_most_common(self, k)
        Finds k words inserted into the trie most often.
    autocomplete(self, prefix)
        Finds the most common word with the given prefix.
    """

    def __init__(self, word_list):
        """Creates the Trie instance, inserts initial words if provided.   
        Parameters
        ----------
        word_list : list
            List of strings to be inserted into the trie upon creation.
        """
        super().__init__()
        self.root = Node(None)

        for word in sorted(word_list):
            self.insert(word)
        
        self.found_words = {}
    
    def insert(self, word):
        import string
        """Inserts a word into the trie, creating missing nodes on the go.
        
        Parameters
        ----------
        word : str
            The word to be inserted into the trie.


        NOTES:
        EDITED FOR QUESTION 4 to increment the counter attribute inside the node whenever a word is repeated
        """
        length = len(word)
        current_node = self.root
        current_node.words += 1
        word = word.lower()

        if word.isalpha():
            repeated = True
            for insertee_char in word:
                #print(current_node.character, current_node.words, children_chars)
                #print(insertee_char)

                #make the current node adopt a new node if the insertee_char isn't already a child
                if insertee_char not in current_node.children_lst:
                    current_node.children[insertee_char] = Node(insertee_char)
                    current_node.children_lst.append(insertee_char)
                    repeated = False
                
                current_node = current_node.children[insertee_char]
                current_node.words += 1
            
            #VERY IMPORTANT:
            # if we didn't create any new node AND THE WORD, not just the terminal node, already exists then our inserted word is repeated!
            if repeated and self.lookup(word):
                #if current_node.character in ["to", "a", "be", "will", "on", "be", "will", "new", "in", "than", "a", "be"]:
                    #print(word, current_node.character, "REPEATED!")
                current_node.count += 1
            #update the terminal of the current word to True when we know that we just finished inserting a word into the tree
            current_node.terminal = True
    
    def lookup(self, word):
        """
        Determines whether a given word is present in the trie.
        
        Parameters
        ----------
        word : str
            The word to be looked-up in the trie.
            
        Returns
        -------
        bool
            True if the word is present in trie; False otherwise.
            
        Notes
        -----
        Your trie should ignore whether a word is capitalized.
        E.g. trie.insert('Prague') should lead to trie.lookup('prague') = True
        """
        word = word.lower()
        current_node = self.root
        word_found = ""

        for char in word:
            children_chars =  list(current_node.children.keys())
            #print(current_node.character, current_node.words, children_chars)
            
            if char in children_chars:
                #print(char, "found")
                current_node = current_node.children[char]
                word_found += current_node.character
            
            elif char not in children_chars:
                #print(char, "one char not found")
                return False
        
        #word_found == word
        return current_node.terminal

    def lookup_prefix(self, prefix):
        """
        checks if a prefix exists in a trie
        """
        prefix = prefix.lower()
        current_node = self.root
        word_found = ""

        for char in prefix:
            children_chars =  list(current_node.children.keys())
            #print(current_node.character, current_node.words, children_chars)
            
            if char in children_chars:
                #print(char, "found")
                current_node = current_node.children[char]
                word_found += current_node.character
            
            elif char not in children_chars:
                #print(char, "one char not found")
                return False
        
        #word_found == word
        return True

    def travie(self, node, prefix):
        """
        recursive preorder traversal
        """
        #break the stack when the child is sterile
        #if node.terminal:
            #print(node.character, "TERMINAL!")

        if len(node.children_lst) < 0:
            return ""
        else:
            if node.character == None:
                self.found_words[prefix] = (node.count)
            else:
                self.found_words[prefix + node.character] = (node.count)
            
            for child_node in node.children.values():
                if node.character == None:
                    self.travie(child_node, prefix)
                else:
                    self.travie(child_node, prefix + node.character)
        
    def clean_found_words(self):
        """
        as the recursive preorder traveral method (travie) returns some words within the tree with repeated and incomplete words, this
        method cleans the found words so that we end up with all the words inserted represented in self.found_words
        """
        #a new dict for the verified words
        verified_words = {}
        
        #sorting the words ***alphabetically*** before parsing them and converting to tuples
        for word in sorted(list(set(self.found_words.keys()))):
            if self.lookup(word):
                verified_words[word] = self.found_words[word]
        
        self.found_words = verified_words

    def k_most_common(self, k):
        """
        Finds k words inserted into the trie most often.

        Parameters
        ----------
        k : int
            Number of most common words to be returned.

        Returns
        ----------
        list
            List of tuples.
            
            Each tuple entry consists of the word and its frequency.
            The entries are sorted by frequency.
        """
        #call the preorder traversal method to update the self.found_words attribute with all the words found while traversing
        self.travie(self.root, "")
        self.clean_found_words()
        
        #raising an error when k is bigger than the number of verified words available
        #if k > len(verified_words):
        #    return "k should be between 1 and len(input)"

        #sorting the verified words by the number of occureneces
        sorted_verified_words = {a: b for a, b in sorted(self.found_words.items(), key=lambda item: item[1], reverse = True)}
        return list(sorted_verified_words.items())[:k]

    def autocomplete(self, prefix):
        """
        Finds the most common word with the given prefix.
        Parameters
        ----------
        prefix : str
            The word part to be “autocompleted”.

        Returns
        ----------
        str
            The complete, most common word with the given prefix.
            
        Notes
        ----------
        The return value is equal to prefix if there is no valid word in the trie.
        The return value is also equal to prefix if prefix is the most common word.
        """
        #converting the prefix input to lower case as all the words in the trie are lower case
        normalized_prefix = prefix.lower()
        if not self.lookup_prefix(normalized_prefix):
            return normalized_prefix
        
        #traversing and cleaning the found strings
        self.travie(self.root, "")
        self.clean_found_words()
        
        prefix_length = len(prefix)
        words_cands = {}
        
        #finding all the words starting with the input prefix
        for cand_word in self.found_words.keys():
            if prefix == cand_word[:prefix_length]:
                words_cands[cand_word] = self.found_words[cand_word]
        
        #return the words with the highest frequency within the words found to begin with the input prefix
        return max(words_cands, key = words_cands.get)

In [ ]:
from requests import get
bad_chars = [';', ',', '.', '?', '!', '1', '2', '3', '4',
             '5', '6', '7', '8', '9', '0', '_', '[', ']']

SH_full = get('http://bit.ly/CS110-Shakespeare').text
SH_just_text = ''.join(c for c in SH_full if c not in bad_chars)
SH_without_newlines = ''.join(c if (c not in ['\n', '\r', '\t']) else " " for c in SH_just_text)
SH_just_words = [word for word in SH_without_newlines.split(" ") if word != ""]

SH_trie = Trie(SH_just_words)

assert SH_trie.autocomplete('hist') == 'history'
assert SH_trie.autocomplete('en') == 'enter'
assert SH_trie.autocomplete('cae') == 'caesar'
assert SH_trie.autocomplete('gen') == 'gentleman'
assert SH_trie.autocomplete('pen') == 'pen'
assert SH_trie.autocomplete('tho') == 'thou'
assert SH_trie.autocomplete('pent') == 'pentapolis'
assert SH_trie.autocomplete('petr') == 'petruchio'

In [ ]:
#I verified this output by inputting each prfix/letter into this tool https://voyant-tools.org/?corpus=abfbde94c4a7cf14cc7ceb50943facc1
#and checking the most popular word that begins with the prefix entered
assert SH_trie.autocomplete("a") == "and"
assert SH_trie.autocomplete("b") == "be"
assert SH_trie.autocomplete("c") == "come"
assert SH_trie.autocomplete("d") == "do"
assert SH_trie.autocomplete("e") == "enter"
assert SH_trie.autocomplete("f") == "for"
assert SH_trie.autocomplete("g") == "good"
assert SH_trie.autocomplete("s") == "so"
assert SH_trie.autocomplete("sh") == "shall"
assert SH_trie.autocomplete("sho") == "should"
assert SH_trie.autocomplete("t") == "the"
assert SH_trie.autocomplete("tho") == "thou"
assert SH_trie.autocomplete("l") == "lord"
assert SH_trie.autocomplete("lov") == "love"